In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

# 원문 데이터 불러오기 (ward에서 쪼개진 클러스터별 데이터 셋)
- 원문 = Ward에서 쪼개진 클러스터별 데이터 셋!
- 클러스터에 맞는 원문 데이터 + 토큰화된 데이터가 필요합니다

In [2]:
df_origin=pd.read_csv('GS건설_ward1_cluster3.csv')
df_origin.head()

,Unnamed: 0,review,target,cluster
0,100,전세집이였던 첫번째 신혼집을 보내주고 두번째 신혼집은 매매로 왔습니다 막간을 이용한...,1,3
1,291,신혼집 랜선집들이 서울 염창역 신혼집을 알아보기 위해서 엄청 많은 부동산을 알아보고...,1,3
2,644,월 결혼 월 신혼집 결정 저희는 올해 월달부터 결혼준비를 시작했어요 결혼 얘기가 나...,1,3
3,645,평범한 신혼집 랜선집들이 저희집을 소개합니다 저희는 재개발을 바라보고 년에 지어진 ...,1,3
4,647,신혼집입주후기 저희는 잔금을 치루고난 후 입주는 주를 기다렸다가 하게되었어요 계약당...,1,3


## 토큰화
- 공출현빈도 코드에서 사용했었던 전처리 코드 그대로 돌려주세요:)

In [29]:
# 토큰화된 데이터가 필요
okt = Okt() 
def preprocess_okt(text):
    #text = spacing(text) #띄어쓰기 보정 위에서 했으면 필요없습니다
    pos_words = okt.pos(text, stem=True)
    words = [word for word, tag in pos_words if tag in ['Noun', 'Adjective', 'Verb'] ]
    
    #여기 stopwords는 이전에 sna노드 만들때 넣었던 단어 그대로 넣어주기
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','아니다',
                 '이렇다','저렇다','도','다','돼다','최고','되다','를','같다',
                 '되었다','오다','가다','으로','자','에','와','한','하다','것',
                 '수','있다','않다','그','이다','이었다', '안','억','이건','정도',
                 '개']
    stopped_words = [w for w in words if not w in stopwords]
    stopped_words=list(set(stopped_words))
    return stopped_words

In [30]:
df_origin['review_token']=df_origin['review'].apply(preprocess_okt)
df_origin['review_token']= [ _ for _ in df_origin.review_token]
df_origin.head()

,Unnamed: 0,review,target,cluster,review_token
0,100,전세집이였던 첫번째 신혼집을 보내주고 두번째 신혼집은 매매로 왔습니다 막간을 이용한...,1,3,"[사당, 강, 말, 당하다, 이용, 주변, 만족하다, 소중하다, 배우다, 고려, 희..."
1,291,신혼집 랜선집들이 서울 염창역 신혼집을 알아보기 위해서 엄청 많은 부동산을 알아보고...,1,3,"[고층, 변, 주변, 광, 들다, 창살, 방지, 소파, 쏠쏠하다, 트루, 하늘, 용..."
2,644,월 결혼 월 신혼집 결정 저희는 올해 월달부터 결혼준비를 시작했어요 결혼 얘기가 나...,1,3,"[겠다, 베란다, 예뻐욤, 주변, 들다, 굴뚝, 고려, 비슷하다, 일, 변동, 부럽..."
3,645,평범한 신혼집 랜선집들이 저희집을 소개합니다 저희는 재개발을 바라보고 년에 지어진 ...,1,3,"[건물, 들다, 부럽다, 선집, 검색, 물, 맞다, 조아하다, 우왕, 만, 구가, ..."
4,647,신혼집입주후기 저희는 잔금을 치루고난 후 입주는 주를 기다렸다가 하게되었어요 계약당...,1,3,"[가계약, 베란다, 시구, 들이다, 결혼식, 어떻다, 정리, 비슷하다, 안방, 부럽..."


# Gephi에서 export한 데이터 불러오기
- gephi 모듈성+아이겐벡터 들어있는 데이터 불러오기 (전체 노드 필요함)
- 이후 모듈성을 고려하여 정의된 세그로 나눠주기

In [31]:
#SNA에서 export한 아이겐벡터
#발견된 세그로 쪼개기
cluster_3=pd.read_csv('아이겐_클러스터3_0.csv')
cluster_3_=pd.read_csv('아이겐_클러스터3_1.csv')

#인테리어
cluster_3_0=cluster_3[cluster_3['modularity_class']==0]
# 경제적고민
cluster_3_1=cluster_3_[cluster_3_['modularity_class']==0]
#인프라
cluster_3_2=cluster_3_[cluster_3_['modularity_class']==1]
#자랑
#cluster_3_3=cluster_3_[cluster_3_['modularity_class']==2]
cluster_3_2

,Id,Label,Degree,modularity_class,eigencentrality
0,가가,가가,212,1,0.162840
2,가깝다,가깝다,3322,1,0.761968
5,가장,가장,1577,1,0.612357
6,강남,강남,520,1,0.310507
8,거리,거리,1896,1,0.644961
...,...,...,...,...,...
4545,치루는데,치루는데,1,1,0.000818
4546,힘들쥬,힘들쥬,1,1,0.000818
4548,하셧을거,하셧을거,1,1,0.000188
4549,환절기,환절기,1,1,0.000188


In [4]:
cluster_3=pd.read_csv('아이겐_클러스터3_0.csv')
cluster_3_=pd.read_csv('아이겐_클러스터3_1.csv')
cluster_3.to_csv('/Users/heojiyeon/Desktop/cluster3_0_gephi data export_지연.csv', encoding='utf-8-sig')
cluster_3_.to_csv('/Users/heojiyeon/Desktop/cluster3_1_gephi data export_지연.csv', encoding='utf-8-sig')

# Cluster별 주요 단어 선정 
- 수기로 작성
- 여러 마이크로 세그먼트에 나올 것으로 예상되는 키워드 선정 지양
- 원문이 엉뚱한 마이크로 세그에 배정될 가능성 존재

In [32]:
# 수기로 키워드 선정하기
# 10개정도 
keywords_3_0=['인테리어','꾸미다','가구', '아기자기하다','조명','분위기','아늑하다','구조','공간', '사진', '예쁘다']
keywords_3_1=['계약','전세','고민','매매','조건','집값','부동산','가격','대출','수리','빌라', '비싸다', '매물', '예산']
keywords_3_2=['위치', '가깝다','편하다', '주변','동네','근처','출퇴근','거리','교통','시설','공원','산책','환경','마트','주차','편의','인프라','놀이터','산책로', '편리하다','지하철']
#keywords_3_3=['깨끗하다','행복', '행복하다','후기','청약', '당첨','크다', '신축','새집', '부럽다', '청약', '당첨', '꽃길']


# 주요 단어를 가지고 eigenvector dictionary만들기
- 리스트에 들어있는 단어들에 아이겐벡터+10 가중치

In [33]:
# eigen 벡터에 +10 가중치 주기
#dictionary 만들기

def add_eigenvector_dict(gephi_dataset, cluster_keywords):
    dict_=defaultdict()
    for w,e in zip(gephi_dataset.Label, gephi_dataset['eigencentrality']):
        if w in cluster_keywords:
            dict_[f'{w}']=e+10
        else:
            dict_[f'{w}']=e

    return dict_

In [34]:
# dictionary 세그별로 만들어주기
dict0=add_eigenvector_dict(cluster_3_0, keywords_3_0)
dict1=add_eigenvector_dict(cluster_3_1, keywords_3_1)
dict2=add_eigenvector_dict(cluster_3_2, keywords_3_2)
#dict3=add_eigenvector_dict(cluster_3_3, keywords_3_3)
dict2

defaultdict(None,
            {'가가': 0.16284,
             '가깝다': 10.761968,
             '가장': 0.612357,
             '강남': 0.310507,
             '거리': 10.644961,
             '걸다': 0.549259,
             '결정': 0.796678,
             '경기도': 0.399204,
             '고덕': 0.04734,
             '고르다': 0.730534,
             '공원': 10.56272,
             '광주': 0.213668,
             '교통': 10.601903,
             '근처': 10.66555,
             '내부': 0.531463,
             '단지': 0.677914,
             '동네': 10.673418999999999,
             '마트': 10.493353,
             '멀다': 0.560207,
             '문래동': 0.129809,
             '버스': 0.390817,
             '산본': 0.202534,
             '산책': 10.540472,
             '산책로': 10.442173,
             '살기': 0.721316,
             '상권': 0.31854,
             '서울': 0.692766,
             '선유도': 0.146541,
             '선택': 0.671544,
             '시간': 0.620734,
             '시설': 10.58282,
             '시장': 0.360575,
             '시흥': 0.178404,
      

# 원문에 dictionary 적용하기
1. token화된 원문에 dictionary 단어가 들어가면 eigenvector를 더해준다
2. 단어가 존재하지 않을 경우 0을 더한다
* dict0, dict1 각각을 원문에 적용하여 eigenvector합을 구한다 

In [35]:
def eigen_sum(df_origin, cluster_dict, column_name):
    '''
    df_origin: 원문데이터
    cluster_dict: 적용하고 싶은 dictionary
    cluster_name: 결과 값이 dataframe으로 나타나는데 여기서 명명할 열이름
    '''
    
    fin_score=[]

    for token_list in df_origin.review_token:
        cluster_score=[] # 리뷰 한 행 넘어갈 때마다 점수 리셋
        for token in token_list:
            if token in cluster_dict.keys():
                for word, eigen in cluster_dict.items():
                    if token == word:
                        #print(token)
                        cluster_score.append(eigen)
                        
            else:
                cluster_score.append(0)#아이겐벡터가 없을 경우 0
        #print(cluster_score)

        # eigen합 구하기
        sum_eigen=np.sum(cluster_score) #아이겐벡터 합        
        
        fin_score.append(sum_eigen)
            
    return pd.DataFrame({column_name: fin_score})  

In [36]:
# 열이름에 해당하는 변수입니다! 원하는 열이름으로 바꿔주세요:)
column_name0='eigen_dict0' 
column_name1='eigen_dict1'
column_name2='eigen_dict2' 
#column_name3='eigen_dict3'


#eigen_sum 함수 적용
#eigen_sum 함수의 인수 바꿔주기
cluster_3_0=eigen_sum(df_origin, dict0, column_name0)
cluster_3_1=eigen_sum(df_origin, dict1, column_name1)
cluster_3_2=eigen_sum(df_origin, dict2, column_name2)
#cluster_3_3=eigen_sum(df_origin, dict3, column_name3)

#데이터합치기
#df_origin.review, df_origin.review_token 두개는 고정!
total_eigen=pd.concat([df_origin.review, df_origin.review_token,
                       cluster_3_0,cluster_3_1, cluster_3_2], axis=1) 
total_eigen

,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2
0,전세집이였던 첫번째 신혼집을 보내주고 두번째 신혼집은 매매로 왔습니다 막간을 이용한...,"[사당, 강, 말, 당하다, 이용, 주변, 만족하다, 소중하다, 배우다, 고려, 희...",105.333112,62.954408,40.048507
1,신혼집 랜선집들이 서울 염창역 신혼집을 알아보기 위해서 엄청 많은 부동산을 알아보고...,"[고층, 변, 주변, 광, 들다, 창살, 방지, 소파, 쏠쏠하다, 트루, 하늘, 용...",109.586556,48.017459,84.273752
2,월 결혼 월 신혼집 결정 저희는 올해 월달부터 결혼준비를 시작했어요 결혼 얘기가 나...,"[겠다, 베란다, 예뻐욤, 주변, 들다, 굴뚝, 고려, 비슷하다, 일, 변동, 부럽...",86.035811,82.893031,27.565324
3,평범한 신혼집 랜선집들이 저희집을 소개합니다 저희는 재개발을 바라보고 년에 지어진 ...,"[건물, 들다, 부럽다, 선집, 검색, 물, 맞다, 조아하다, 우왕, 만, 구가, ...",86.999359,11.942628,0.884540
4,신혼집입주후기 저희는 잔금을 치루고난 후 입주는 주를 기다렸다가 하게되었어요 계약당...,"[가계약, 베란다, 시구, 들이다, 결혼식, 어떻다, 정리, 비슷하다, 안방, 부럽...",99.377565,62.399100,6.102810
...,...,...,...,...,...
1997,반가워요 우연히 지인통해서 레몬테라스 알게되서 가입하게 되었어요 좋은 정보 서로 공...,"[저, 테라스, 통해, 알, 되어다, 좋다, 가입, 서로, 반갑다, 공유, 정보, ...",0.977501,0.000000,0.000000
1998,가입했어요 레몬테라스 그동안 들어와보고 싶었던 곳이었어요 회원님들과 따뜻한 소통 공...,"[싶다, 보고, 따뜻하다, 장소, 테라스, 되어다, 좋다, 소통, 가입, 회원, 들...",1.851688,0.000000,0.176521
1999,첫 인사드려요 늘 알고 있던 레몬테라스인데 이제야 가입을 했어요 잘 부탁드립니다 잘...,"[늘, 인사, 부탁드리다, 알, 테라스, 자다, 가입, 첫, 반갑다, 레몬, 지내다...",1.075454,0.000000,0.000000
2000,맘카페 대표 레몬테라스 잘 반가워요 반갑습니다 반갑습니다 반갑습니다 반갑습니다,"[카페, 맘, 테라스, 자다, 반갑다, 대표, 레몬]",1.576530,0.000000,0.000000


# 최종 클러스터 설정

### eigen_dict가 모두 0일때 --> 모든 클러스터에 넣어준다 
- 데이터를 불려주기 위함

In [37]:
# dictionary점수 모든 열이 0인 값 vs. 0이 아닌값을 분리
zero_rows = total_eigen[(total_eigen['eigen_dict0'] == 0)&(total_eigen['eigen_dict1'] == 0) & (total_eigen['eigen_dict2'] == 0)]
non_zero_rows = total_eigen[~((total_eigen['eigen_dict0'] == 0)&(total_eigen['eigen_dict1'] == 0) & (total_eigen['eigen_dict2'] == 0))]

max_column = non_zero_rows[['eigen_dict0', 'eigen_dict1', 'eigen_dict2']].idxmax(axis=1)
non_zero_rows['final']=[_[-1:] for _ in max_column]

# none값 모든 seg에 추가
none0=zero_rows.copy()
none1=zero_rows.copy()
none2=zero_rows.copy()

none0['final']=0
none1['final']=1
none2['final']=2

#seg 0
seg0=non_zero_rows[non_zero_rows['final']=='0']
final_seg_0=pd.concat([seg0, none0])
display(final_seg_0)
print(final_seg_0.shape)

#seg1
seg1=non_zero_rows[non_zero_rows['final']=='1']
final_seg_1=pd.concat([seg1, none1])
display(final_seg_1)
print(final_seg_1.shape)

#seg2
seg2=non_zero_rows[non_zero_rows['final']=='2']
final_seg_2=pd.concat([seg2, none2])
display(final_seg_2)
print(final_seg_2.shape)

#csv파일로 저장
final_seg_0.to_csv('cluster3_seg0_인테리어.csv', encoding='utf-8-sig')
final_seg_1.to_csv('cluster3_seg2_경제적고민.csv',encoding='utf-8-sig')
final_seg_2.to_csv('cluster3_seg1_인프라.csv',encoding='utf-8-sig')

/var/folders/sf/g5ffdfm57wv22zjhcdsjt1rm0000gn/T/ipykernel_19815/5566846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_zero_rows['final']=[_[-1:] for _ in max_column]


,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,final,final_seg
0,전세집이였던 첫번째 신혼집을 보내주고 두번째 신혼집은 매매로 왔습니다 막간을 이용한...,"[사당, 강, 말, 당하다, 이용, 주변, 만족하다, 소중하다, 배우다, 고려, 희...",105.333112,62.954408,40.048507,0,NaN
1,신혼집 랜선집들이 서울 염창역 신혼집을 알아보기 위해서 엄청 많은 부동산을 알아보고...,"[고층, 변, 주변, 광, 들다, 창살, 방지, 소파, 쏠쏠하다, 트루, 하늘, 용...",109.586556,48.017459,84.273752,0,NaN
2,월 결혼 월 신혼집 결정 저희는 올해 월달부터 결혼준비를 시작했어요 결혼 얘기가 나...,"[겠다, 베란다, 예뻐욤, 주변, 들다, 굴뚝, 고려, 비슷하다, 일, 변동, 부럽...",86.035811,82.893031,27.565324,0,NaN
3,평범한 신혼집 랜선집들이 저희집을 소개합니다 저희는 재개발을 바라보고 년에 지어진 ...,"[건물, 들다, 부럽다, 선집, 검색, 물, 맞다, 조아하다, 우왕, 만, 구가, ...",86.999359,11.942628,0.884540,0,NaN
4,신혼집입주후기 저희는 잔금을 치루고난 후 입주는 주를 기다렸다가 하게되었어요 계약당...,"[가계약, 베란다, 시구, 들이다, 결혼식, 어떻다, 정리, 비슷하다, 안방, 부럽...",99.377565,62.399100,6.102810,0,NaN
...,...,...,...,...,...,...,...
1941,새로 가입했어요 반갑습니당 반갑습니다 반갑습니다,"[습, 가입, 니당다, 새롭다, 반갑다]",0.000000,0.000000,0.000000,NaN,0.0
1962,반갑습니다 레몬테라스 가입했어요 잘부탁드립니다 반갑습니다 반갑습니다,"[부탁드리다, 테라스, 반갑다, 레몬, 가입]",0.000000,0.000000,0.000000,NaN,0.0
1976,반갑습니다 드디어 레몬테라스에 왔네요 반갑습니다 반갑습니다아 반갑습니다 안녕하세욤 ...,"[테라스, 습, 반갑다, 니당다, 욤, 안녕하다, 레몬]",0.000000,0.000000,0.000000,NaN,0.0
1992,반가워요 레몬테라스에 가입했어요 반갑습니다 반가워용 반가워요 반가워요 반갑습니다 반...,"[테라스, 반, 반갑다, 안녕하다, 레몬, 가입]",0.000000,0.000000,0.000000,NaN,0.0


(1175, 7)


,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,final,final_seg
6,신혼집 결정 및 결정후기입니다 신혼집 결정 및 결정 후기를 작성해보려해요 결혼을 생...,"[고층, 가능성, 사내, 모든, 들다, 고려, 산도, 층, 부럽다, 초, 되어다, ...",6.249867,124.848154,60.778212,1,NaN
7,신혼집 결정 및 결정이유 제일 머리아픈 게 신혼집 같아요 요새 집값도 너무 올라버려...,"[스드메, 이유, 불끈, 주변, 들다, 고려, 일, 부럽다, 되어다, 경기, 방어,...",53.213745,85.150090,71.646465,1,NaN
13,신혼집 결정 및 결정이유 월 예신입니다 결혼준비할 때 집이 제일 큰 산 이라고 해서...,"[이유, 값, 만족하다, 주변, 들다, 고려, 낮, 오른, 신입, 미치다, 검색, ...",19.610674,105.778061,51.498310,1,NaN
32,신혼집 결정 이유에요 여러분 다이렉트 웨딩박람회 가서 플래너님 만나 설명듣고 스튜디...,"[울다, 탓, 이유, 돌아오다, 말, 말다, 이용, 물건, 주변, 들다, 만족하다,...",40.096920,102.183676,83.520888,1,NaN
39,신혼집 결정 및 결정이유 입니다 예비 신랑 입니다 결혼 준비를하면서 가장 큰돈이 들...,"[이유, 모든, 건물, 들다, 중간, 고려, 평수, 여자친구, 새롭다, 촌, 부엌,...",11.608285,80.353154,72.290991,1,NaN
...,...,...,...,...,...,...,...
1941,새로 가입했어요 반갑습니당 반갑습니다 반갑습니다,"[습, 가입, 니당다, 새롭다, 반갑다]",0.000000,0.000000,0.000000,NaN,1.0
1962,반갑습니다 레몬테라스 가입했어요 잘부탁드립니다 반갑습니다 반갑습니다,"[부탁드리다, 테라스, 반갑다, 레몬, 가입]",0.000000,0.000000,0.000000,NaN,1.0
1976,반갑습니다 드디어 레몬테라스에 왔네요 반갑습니다 반갑습니다아 반갑습니다 안녕하세욤 ...,"[테라스, 습, 반갑다, 니당다, 욤, 안녕하다, 레몬]",0.000000,0.000000,0.000000,NaN,1.0
1992,반가워요 레몬테라스에 가입했어요 반갑습니다 반가워용 반가워요 반가워요 반갑습니다 반...,"[테라스, 반, 반갑다, 안녕하다, 레몬, 가입]",0.000000,0.000000,0.000000,NaN,1.0


(251, 7)


,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,final,final_seg
10,신혼집 결정 및 결정이유입니다 신혼집 결정 및 이유 후기 올립니다 저희는 남친이랑 ...,"[목적, 나쁘다, 이유, 산책, 말, 주변, 들다, 핫, 고, 오빠, 매일, 부럽다...",29.797485,25.757510,49.056135,2,NaN
14,직방으로 알아본 신혼집찾기 동탄 자이파밀리에 후 결혼하는거 이렇게 어려운건지 몰랐어...,"[다양하다, 가능성, 허니문, 수원, 부럽다, 단지, 놀다, 되어다, 검색, 초, ...",32.391885,47.397958,82.795568,2,NaN
17,신혼집 결정이유입니다 월 일 식 준비중인 잉샤입니다 오늘은 저희 신혼집 결정이유에 ...,"[산책, 이유, 주변, 중간, 대해, 들다, 이루어지다, 빗, 공정, 일, 산책로,...",38.678751,14.696793,83.798765,2,NaN
22,신혼집 결정 및 결정 이유입니다 다음달에 예식 예정인 예비신부 연금술사 입니다 신혼...,"[산책, 이유, 빽빽하다, 말, 주변, 들다, 이면, 친구, 정리, 산책로, 부럽다...",7.347807,15.170946,149.773522,2,NaN
23,신혼집 결정 및 결정 이유입니다 신혼집 결정 및 결정한 이유를 얘기해보려고 해요 처...,"[이유, 빽빽하다, 말, 주변, 들다, 필요하다, 치루셨어, 부럽다, 환경, 푸릇푸...",33.717081,42.807609,95.866247,2,NaN
...,...,...,...,...,...,...,...
1941,새로 가입했어요 반갑습니당 반갑습니다 반갑습니다,"[습, 가입, 니당다, 새롭다, 반갑다]",0.000000,0.000000,0.000000,NaN,2.0
1962,반갑습니다 레몬테라스 가입했어요 잘부탁드립니다 반갑습니다 반갑습니다,"[부탁드리다, 테라스, 반갑다, 레몬, 가입]",0.000000,0.000000,0.000000,NaN,2.0
1976,반갑습니다 드디어 레몬테라스에 왔네요 반갑습니다 반갑습니다아 반갑습니다 안녕하세욤 ...,"[테라스, 습, 반갑다, 니당다, 욤, 안녕하다, 레몬]",0.000000,0.000000,0.000000,NaN,2.0
1992,반가워요 레몬테라스에 가입했어요 반갑습니다 반가워용 반가워요 반가워요 반갑습니다 반...,"[테라스, 반, 반갑다, 안녕하다, 레몬, 가입]",0.000000,0.000000,0.000000,NaN,2.0


(588, 7)
